# Accessing NeuroHub using the CBRAIN API

## 0. Set up boilerplate

You **must** make sure the module `httpproxy/1.0` is loaded in your environment before starting this notebook on [JupyterLab on Beluga](https://jupyterhub.beluga.computecanada.ca/). **Do not** load the module in your `.bashrc`, it interferes with login node, this is needed for the compute nodes only.

Let's first import libraries that are required to send HTTP requests and manipulate JSON data.

In [ ]:
import requests
import json
import getpass

We define the API endpoint. See the [CBRAIN API](https://portal.cbrain.mcgill.ca/swagger#/) for more information about how to use the API.

In [ ]:
base_url = 'https://portal.cbrain.mcgill.ca'

## 1. Connect to NeuroHub

Connect to NeuroHub with a username and a password.  See the [NeuroHub Portal](https://neurohub.ca/portal.html) for information about how to register for an account.

We could use token-based authentication: login to the [NeuroHub Portal](https://neurohub.ca/portal.html) to generate a token.

Here we'll use username/password authentication where we `POST` our `credentials` to `/session` and get a token that will need to be sent as a parameter within each subsequent request. The token response is stored in `session_info`.

In [ ]:
credentials = {
    'login': input('username'),
    'password': getpass.getpass('password')
}

response = requests.post(
    url = '/'.join([base_url, 'session']),
    data = credentials,
    headers = {'Accept': 'application/json'}
)

if response.status_code != requests.codes.ok:
    print('Login failed.')
    print(response)
else:
    session_info = response.json()
    print('Login successful.')
    print(session_info['user_id'])

## 2. Get all user-accessible files

We `GET` our `request` from `/userfiles` for a list of all the files we have access to. This may be many thousands of files and ***may take many minutes***. 

In [ ]:
try:

    user_id = str(session_info['user_id'])
    cbrain_api_token = session_info['cbrain_api_token']

except NameError:
    raise UserWarning('Login first.')
 
files = []

request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    response = requests.get(
        url = '/'.join([base_url, 'userfiles']),
        data = request,
        headers = {'Accept': 'application/json'}
    )

    if response.status_code != requests.codes.ok:
        print('User files request failed.')
        print(response)
        break

    # Collect the responses on this page then increment
    files += response.json()
    request['page'] += 1
    
    # Stop requesting responses when we're at the last page
    if len(response.json()) < request['per_page']:
        break 
    
print("{} files found\n".format(str(len(files))))

## 3. Example usage
### 3.a Filter files
Files can be filtered on any of their properties.  Here we filter for a name that contains `mnc`.

In [ ]:
result = list(filter(lambda f: 'mnc' in f['name'], files))
print(json.dumps(result, indent=4, sort_keys=True))

### 3.b Get file contents

We `GET` the user file content from `/userfiles/{id}/content` for the first filtered result.

In [ ]:
response = requests.get(
    url = '/'.join([base_url, 'userfiles', str(result[0]['id']), 'content']),
    data = {'cbrain_api_token': cbrain_api_token},
    headers = {'Accept': 'application/json'}
)

print(response.headers)

### 3.c Process data

You can now do any interactive processing with `response.content`.

Let's compare the computed size to the stated size.

In [ ]:
print(len(response.content))
print(result[0]['size'])